In [1]:
from IPython.core.display import HTML
with open('../style.css', 'r') as file:
    css = file.read()
HTML(css)

# The Knight's Tour

This notebook computes a solution to the [knight's tour](https://en.wikipedia.org/wiki/Knight%27s_tour) using the constraint solver `Z3`.  Furthrmore, we will compute a closed tour, i.e. the knight will return to its starting square in its last move.

In [2]:
import z3

Given an integer from the set $\{0, 1, \cdots, 64\}$, the function `row(i)` computes the name of the variable that specifies the *row* of the knight after its $i^{\textrm{th}}$ move.

In [3]:
def row(i):
    return f'R{i}'

Given an integer from the set $\{0, 1, \cdots, 64\}$, the function `row(i)` computes the name of the variable that specifies the *column* of the knight after its $i^{\textrm{th}}$ move.

In [4]:
def col(i):
    return f'C{i}'

The function `all_variables` computes the names of all variables.

In [5]:
def all_variables():
    Variables = set()
    for i in range(64+1):
        Variables.add(row(i))
        Variables.add(col(i))
    return Variables

The function `is_knight_move(i)` returns a formula that specifies that the $i^{\textrm{th}}$ move is a legal move for a knight.
In order to form the *conjunction* of two formulas we use the function `z3.And`, while the *disjunction* is build with the function `z3.Or`.

The figure below shows the moves of a knight:  The knight on `e4` can jump to all red squares.
<img src="knight-moves.png" width="50%">

In [6]:
def is_knight_move(i):
    r  = row(i)
    c  = col(i)
    rX = row(i+1)
    cX = col(i+1)
    Formulas = set()
    for delta_r, delta_c in [(1, 2), (2, 1)]:
        Formulas.add(f'z3.And({rX} == {r} + {delta_r}, {cX} == {c} + {delta_c})')
        Formulas.add(f'z3.And({rX} == {r} + {delta_r}, {cX} + {delta_c} == {c})')
        Formulas.add(f'z3.And({rX} + {delta_r} == {r}, {cX} == {c} + {delta_c})')
        Formulas.add(f'z3.And({rX} + {delta_r} == {r}, {cX} + {delta_c} == {c})') 
    return 'z3.Or(' + ', '.join(Formulas) + ')'

The function `all_different` computes a set of formulas specifiying that
$$ \langle \textrm{R}i, \textrm{C}i \rangle \not= \langle \textrm{R}j, \textrm{C}j \rangle $$
provided that $i \not= j$.  This specifies that, with the exception of the starting square, the knight will not not visit a square twice.  This way, we ensure that all squares are visited as the tour has a length of 64 and there are exactly $8 \times 8 = 64$ squares on a chessboard.

In [7]:
def all_different():
    Result = set()
    for i in range(62+1):
        for j in range (i+1, 63+1):
            Result.add(f'z3.Or({row(i)} != {row(j)}, {col(i)} != {col(j)})')
    return Result

In [8]:
def all_constraints():
    Constraints = all_different()
    Constraints.add(f'{row(0 )} == 0')
    Constraints.add(f'{col(0 )} == 0')
    Constraints.add(f'{row(64)} == 0')
    Constraints.add(f'{col(64)} == 0')
    for i in range(63+1):
        Constraints.add(is_knight_move(i))
    for i in range(64+1):
        Constraints.add(f'{row(i)} >= 0')
        Constraints.add(f'{col(i)} >= 0')
    return Constraints

In [9]:
len(all_constraints())

2214

The function `solve(Constraints, Variables)` receives two arguments:
- `Constraints` is a set of formulas representing a constraint satisfaction problem.
- `Variables`   is the set of variables that occur in this formulas.

   It is assumed that all variables can be presented by bit-vector of length 4.
   We need 4 bits because the numbers use a sign bit.

The function computes a solution to the given problem and returns this solution.

In [10]:
def solve(Constraints, Variables):
    Environment = {}
    exec('import z3', Environment)
    for v in Variables:
        exec(f'{v} = z3.BitVec(f"{v}", 4)', Environment)
    s = z3.Solver()
    for c in Constraints:
        s.add(eval(c, Environment))
    result = str(s.check())
    if result == 'sat':
        m = s.model()
        S = { v: m[eval(v, Environment)] for v in Variables }
        return S
    elif result == 'unsat':
        print('The problem is not solvable.')
    else:
        print('Z3 cannot determine whether the problem is solvable.')

Unfortunately, the execution times of the following cell vary greatly. 
Sometimes it takes 7 minutes, sometimes 35 minutes.

In [11]:
%%time
Solution = solve(all_constraints(), all_variables())
Solution

CPU times: user 23min 9s, sys: 2.71 s, total: 23min 11s
Wall time: 23min 12s


{'C47': 6,
 'C56': 7,
 'C64': 0,
 'R46': 6,
 'R28': 7,
 'R31': 3,
 'C24': 0,
 'C61': 0,
 'C42': 6,
 'C2': 3,
 'R30': 4,
 'C32': 2,
 'C34': 0,
 'C26': 3,
 'R57': 3,
 'R17': 3,
 'C21': 1,
 'C25': 1,
 'R1': 1,
 'C5': 6,
 'C59': 3,
 'C38': 6,
 'R25': 6,
 'C39': 7,
 'C3': 5,
 'C48': 7,
 'R42': 6,
 'R61': 5,
 'C30': 6,
 'C43': 4,
 'R12': 2,
 'R55': 0,
 'R49': 6,
 'R54': 2,
 'C27': 4,
 'C18': 1,
 'R9': 1,
 'C50': 7,
 'C17': 0,
 'R59': 6,
 'R43': 7,
 'R37': 2,
 'R52': 5,
 'C12': 6,
 'C54': 4,
 'R27': 5,
 'C8': 1,
 'R53': 4,
 'C10': 2,
 'R5': 1,
 'R51': 7,
 'R7': 2,
 'R32': 2,
 'C52': 5,
 'R64': 0,
 'C46': 4,
 'C19': 0,
 'R47': 5,
 'R14': 1,
 'C57': 6,
 'C1': 2,
 'C4': 7,
 'R8': 3,
 'C41': 7,
 'C35': 2,
 'R10': 0,
 'R23': 5,
 'C55': 5,
 'C58': 4,
 'R58': 4,
 'R56': 1,
 'C33': 1,
 'R63': 2,
 'R60': 7,
 'C9': 0,
 'C45': 2,
 'C53': 3,
 'C49': 5,
 'C37': 5,
 'C36': 3,
 'C7': 3,
 'R3': 4,
 'R15': 0,
 'C6': 4,
 'C0': 0,
 'C31': 4,
 'C40': 5,
 'C15': 3,
 'R39': 2,
 'R45': 7,
 'C44': 3,
 'R2': 3,
 'R16

The function `create_board(Solution)` returns a matrix `Board` of size $8\times 8$.
The following holds:
$$ \texttt{Board}[\texttt{R}i][\texttt{C}i] = i $$
Therefore, if `Board[r][c] == i`, then at the beginning of the $i^{\textrm{th}}$ move the knight is located in row `r` and column `c`. 

In [12]:
def create_board(Solution):
    Board = [[0 for _ in range(8)] for _ in range(8)]
    for i in range(1, 65):
        r = Solution[row(i)].as_long()
        c = Solution[col(i)].as_long()
        Board[r][c] = i
    return Board

In [13]:
create_board(Solution)

[[64, 33, 10, 15, 6, 55, 38, 13],
 [9, 16, 1, 36, 11, 14, 5, 56],
 [34, 63, 32, 7, 54, 37, 12, 39],
 [17, 8, 35, 2, 31, 40, 57, 4],
 [24, 21, 62, 53, 58, 3, 30, 41],
 [61, 18, 23, 44, 27, 52, 47, 50],
 [22, 25, 20, 59, 46, 49, 42, 29],
 [19, 60, 45, 26, 43, 28, 51, 48]]

The function `print_board` prints the given `Board`.

In [14]:
def print_board(Board):
    n = len(Board)
    # Determine the width of the widest element in the matrix
    width = max([ len(str(element)) for row in Board
                                    for element in row
                ])
    # Create the top and bottom of the matrix
    top_line = '╔'
    for i in range(n - 1):
        top_line += '═' * (width + 2) + '╦'
    top_line += '═' * (width + 2) + '╗'
    mid_line = '╠'
    for i in range(n - 1):
        mid_line += '═' * (width + 2) + '╬'
    mid_line += '═' * (width + 2) + '╣'    
    bot_line = '╚'
    for i in range(n - 1):
        bot_line += '═' * (width + 2) + '╩'
    bot_line += '═' * (width + 2) + '╝'
    # Print the top of the matrix
    print(top_line)
    # Iterate through the rows and columns of the matrix, and print
    # each element with proper padding
    for i, row in enumerate(Board):
        line = '\u2551'
        for element in row:
            line += f' {element:>{width}} ║'
        print(line)
        # Print a horizontal line
        if i < len(Board) - 1:
            print(mid_line)
    # Print the bottom of the matrix
    print(bot_line)

In [15]:
print_board(create_board(Solution))

╔════╦════╦════╦════╦════╦════╦════╦════╗
║ 64 ║ 33 ║ 10 ║ 15 ║  6 ║ 55 ║ 38 ║ 13 ║
╠════╬════╬════╬════╬════╬════╬════╬════╣
║  9 ║ 16 ║  1 ║ 36 ║ 11 ║ 14 ║  5 ║ 56 ║
╠════╬════╬════╬════╬════╬════╬════╬════╣
║ 34 ║ 63 ║ 32 ║  7 ║ 54 ║ 37 ║ 12 ║ 39 ║
╠════╬════╬════╬════╬════╬════╬════╬════╣
║ 17 ║  8 ║ 35 ║  2 ║ 31 ║ 40 ║ 57 ║  4 ║
╠════╬════╬════╬════╬════╬════╬════╬════╣
║ 24 ║ 21 ║ 62 ║ 53 ║ 58 ║  3 ║ 30 ║ 41 ║
╠════╬════╬════╬════╬════╬════╬════╬════╣
║ 61 ║ 18 ║ 23 ║ 44 ║ 27 ║ 52 ║ 47 ║ 50 ║
╠════╬════╬════╬════╬════╬════╬════╬════╣
║ 22 ║ 25 ║ 20 ║ 59 ║ 46 ║ 49 ║ 42 ║ 29 ║
╠════╬════╬════╬════╬════╬════╬════╬════╣
║ 19 ║ 60 ║ 45 ║ 26 ║ 43 ║ 28 ║ 51 ║ 48 ║
╚════╩════╩════╩════╩════╩════╩════╩════╝


# Visualization

If you have not yet installed `chess-problem-visuals` you have to uncomment the following line.

In [16]:
!pip install git+https://github.com/reclinarka/chess-problem-visuals

  Cloning https://github.com/reclinarka/chess-problem-visuals to /private/var/folders/zg/r2ldnzsn677_p_qb62_kt1j40000gn/T/pip-req-build-_dgoqwvu
  Running command git clone --filter=blob:none --quiet https://github.com/reclinarka/chess-problem-visuals /private/var/folders/zg/r2ldnzsn677_p_qb62_kt1j40000gn/T/pip-req-build-_dgoqwvu
  Resolved https://github.com/reclinarka/chess-problem-visuals to commit 764a29b376fe9dd3cbb2623ce8740f73c6711fa4
  Preparing metadata (setup.py) ... done


In [17]:
from chess_problem_visuals import problem_board

The function `show_solution` displays the given solution on a chessboard.
The solution `Board` is represented as a list of lists.  We have `Board[row][col] == k` if the $k^\textrm{th}$ move leads the knight to the position `(row, col)`.

In [18]:
def show_solution(Board, width="50%"):
    n         = len(Board)
    Positions = {}
    for row in range(n):
        for col in range(n):
            k = Board[row][col]
            Positions[k] = (row, col)
    start = (0, 0)
    Path  = [start]
    for k in range(1, n*n+1):
        Path.append(Positions[k])
    Visual = problem_board(n, K_start=start, K_path=Path, 
                           html_width=width, 
                           arrow_color="darkgreen",
                           arrow_width=0.2)
    return Visual

In [19]:
show_solution(create_board(Solution))